# Tutorial

This tutorial will walk you through the main features of atoti by creating an application to analyze the sales of a company.

We'll see how to:

- Load normalized data in multiple tables to create a multidimensionnal cube.
- Define aggregated measures to provide application-specific and high-level insights.
- Build no-code interactive charts and tables in JupyterLab.
- Create dashboards in the built-in web app.

We encourage you to play this notebook in JupyterLab:

- Copy the tutorial: `python -m atoti.copy_tutorial tutorial`
- Open JupyterLab: `jupyter lab`
- Open the created **tutorial** directory and start playing its notebook.

For more information about a functionality, you can use the API reference or <kbd>Shift ⇧</kbd> + <kbd>Tab ⇥</kbd> on a Python symbol in JupyterLab.

## Getting started

### From CSV to Cube

In this part of the tutorial, you will create your first cube from a CSV file and learn multidimensionnal concepts such as _cube_, _dimension_, _hierarchy_, _measure_.

Let's start by creating a _session_:

In [ ]:
import atoti as tt

session = tt.create_session()

We can now load the data from a CSV file into an in-memory table called a _table_:

In [ ]:
sales_table = session.read_csv("data/sales.csv", keys=["Sale ID"])

We can have a look at the loaded data.
They are sales from a company:

In [ ]:
sales_table.head()

We will come back to tables in details later, for now we will use the one we have to create a _cube_:

In [ ]:
cube = session.create_cube(sales_table)

That's it, you have created your first cube!
But what's a cube exactly and how to use it?

### Multidimensional concepts

A _cube_ is a multidimensional view of some data, making it easy to explore, aggregate, filter and compare.
It's called a cube because each attribute of the data can be represented as a dimension of the cube:

<img alt="Multidimensional cube concept" src="images/olap/cube-concept.svg" width="400" />

The axes of the cube are called _hierarchies_.
The purpose of multidimensionnal analysis is to visualize some numeric indicators at specific coordinates of the cube.
These indicators are called _measures_.
An example of measure would be the quantity of products sold.

We can list the hierarchies in our cube:

In [ ]:
# Aliasing the hierarchies property to a shorter variable name because we will use it a lot.
h = cube.hierarchies
h

The cube has automatically created a hierarchy for each non numeric column: **Date**, **Product**, **Sale ID** and **Shop**.

You can see that the hierarchy are grouped into dimensions.
Here we have a single dimension called sales, which is the name of the table the columns come from. We will see how to move hierarchies between dimensions later.

Hierarchies are also made of _levels_.
Levels of the same hierarchy are attributes with a parent-child relationship.
For instance, a city belongs to a country so **Country** and **City** could be the two levels of a **Geography** hierarchy.

At the moment, to keep it simple, we only have single-level hierarchies.

In [ ]:
l = cube.levels

Let's have a look at the measures of the cube that have been inferred from the data:

In [ ]:
m = cube.measures
m

The cube has automatically created the _sum_ and _mean_ aggregations for all the numeric columns of the dataset.

Note that a measure isn't a single result number, it's more a formula that can be evaluated for any coordinates of the cube.

For instance, we can query the _grand total_ of **Quantity.SUM**, which means summing the sold quantities over the whole dataset:

<img alt="Grand total" src="images/olap/grand-total.svg" width="400" />

In [ ]:
cube.query(m["Quantity.SUM"])

But we can also _dice_ the cube to get the quantity for each **Shop**, which means taking one _slice_ of the cube for each **Shop**:

<img alt="Dicing the cube" src="images/olap/slices.svg" width="400" />

In [ ]:
cube.query(m["Quantity.SUM"], levels=[l["Shop"]])

We can _slice_ on a single **Shop**:

<img alt="Slicing the cube" src="images/olap/slice.svg" width="400" />

In [ ]:
cube.query(
    m["Quantity.SUM"],
    condition=l["Shop"] == "shop_0",
)

We can dice along 2 different axes and take the quantity per product and date.

<img alt="Pivot table" src="images/olap/pivot-table.svg" width="400" />

In [ ]:
cube.query(m["Quantity.SUM"], levels=[l["Date"], l["Product"]])

We can even combine these operations to slice on one hierarchy and dice on the two others:

<img alt="Slice and dice" src="images/olap/slice-and-dice.svg" width="400" />

In [ ]:
cube.query(
    m["Quantity.SUM"],
    levels=[l["Date"], l["Product"]],
    condition=l["Shop"] == "shop_0",
)

### First visualization

So far we have used [cube.query()](../lib/atoti.cube.rst#atoti.cube.Cube.query) which returns a table as a pandas DataFrame but a better way to visualize multidimensional data is a pivot table.
With atoti's JupyterLab extension, you can do advanced and interactive visualizations such as pivot tables and charts directly into your notebook by calling [session.visualize()](../lib/atoti.session.rst#atoti.session.Session.visualize).

This will create a widget and open the atoti tab on the left with tools to manipulate the widget.

Let's start by creating a pivot table:

- Run `session.visualize()`.
- In the left panel, click on a measure such as **Quantity.SUM** to add it.
- Click on a hierarchy such as **Date** to get the quantity per date.
- Drag and drop another hierarchy such as **Product** to the **Columns** section to get the quantity sold per day and per product.

![First pivot table](images/gifs/first-pivot-table.gif)

In [ ]:
session.visualize()

A pivot table can be switched to a chart.

For instance let's switch to a simple line chart.

![First atoti chart](images/gifs/first-chart.gif)

In [ ]:
session.visualize()

### Drilldown and filters

Multidimensional analysis is meant to be done from top to bottom: start by visualizing the indicators at the top level then drilldown to explain the top figures with more details.

For instance, we can visualize some measures per date then drilldown on **Shop** for a specific date, then see the products sold by a specific shop on this date.

Using the previous cube representation, this is like zooming more and more on a part of the cube.

<img alt="Drilldown the cube" src="images/olap/drilldown.svg" width="1200" />

![Drilldown](images/gifs/drilldown.gif)

In [ ]:
session.visualize()

Hierarchies can be filtered when building widgets.
Let's apply a filter on the previous chart and only visualize the quantity for a group of selected products.

![Chart filter](images/gifs/filter-chart.gif)

In [ ]:
session.visualize()

### Dashboarding application

Being able to quickly build widgets inside a notebook without coding is nice to rapidly explore the data, iterate on your model and share some results.
However, to provide richer insights, dashboards are even better.
That's why atoti comes with a web application that can be accessed outside of the notebook and where widgets can be laid out to form dashboards.

The application can be accessed with this link:

In [ ]:
session.link()

It's possible to publish widgets built in the notebook to the application by right clicking on them and selecting "Publish widget in app".
They will then be available in the "Saved widgets" section.

![Publish widget in application](images/gifs/open-in-app.gif)

## Enriching the cube

In the previous section, you have learned how to create a basic cube and manipulate it.
We will now enrich this cube with additional attributes and more interesting measures.

### Join

Currently, we have very limited information about our products: only the ID.
We can load a CSV containing more details into a new table:

In [ ]:
products_table = session.read_csv("data/products.csv", keys=["Product"])

Note that a table can have a set of keys.
These keys are the columns which make each line unique.
Here, it's the product ID.

If you try to insert a new row with the same keys as an existing row, it will override the existing one.

In [ ]:
products_table.head()

This table contains the category, subcategory, size, color, purchase price and brand of the product.
Both tables have a **Product** column we can use to [join](../lib/atoti.table.rst#atoti.table.Table.join) them.

Note that this is a database-like join and not a pandas-like join.
All the details from `products_table` won't be inlined into `sales_table`.
Instead, this just declares a reference between these two tables that the cube can use to provide more analytical axes.

In [ ]:
sales_table.join(products_table, mapping={"Product": "Product"})

You can visualize the structure of the session's tables:

In [ ]:
session.tables.schema

The new columns have been automatically added to the cube as hierarchies, in a dimension with the same name as the new table:

In [ ]:
h

You can use them directly in a new widget.
For instance, let's create a bar chart to visualize the mean price per subcategory of product:

![Price per category](images/gifs/price-per-category.png)

In [ ]:
session.visualize()

We can also make a donut chart to see how all the sales are distributed between brands:

![Donut chart brands](images/gifs/donut-chart.png)

In [ ]:
session.visualize()

Note that after the join we can add a new measure called **Purchase price.VALUE** based on the corresponding column of the joined table.
This measure represents the value of the column so it is only defined when all the keys of the joined table are expressed in the query.

In [ ]:
m["Purchase price.VALUE"] = tt.value(products_table["Purchase price"])

For instance we can check the purchase price per **Product**:

In [ ]:
cube.query(m["Purchase price.VALUE"], levels=[l["Product"]])

In a similar way, we can enrich the data about the shops:

In [ ]:
shops_table = session.read_csv("data/shops.csv", keys=["Shop ID"])
shops_table.head()

In [ ]:
sales_table.join(shops_table, mapping={"Shop": "Shop ID"})
session.tables.schema

### New measures

So far we have only used the default measures which are basic aggregations of the numeric columns.
We can add new custom measures to our cube.

#### Max

We'll start with a simple aggregation taking the maximum price of the sales table:

In [ ]:
m["Max price"] = tt.agg.max(sales_table["Unit price"])

This new measure is directly available:

In [ ]:
cube.query(m["Max price"], include_totals=True, levels=[l["Category"]])

#### Fact-level operations

To compute aggregates based of data which comes directly from the columns of a table, you can pass the calculation directly to the desired aggregation function. This is more efficient than first converting the columns to measures before the aggregation.

Let's use this to compute the total amount earned from the sale of the products, as well as the average.

Note the use of the `origin` scope instructing to perform the multiplication for each **Sale ID** and then do the sum.

In [ ]:
m["Amount.SUM"] = tt.agg.sum(
    sales_table["Quantity"] * sales_table["Unit price"],
    scope=tt.scope.origin(l["Sale ID"]),
)
m["Amount.MEAN"] = tt.agg.mean(
    sales_table["Quantity"] * sales_table["Unit price"],
    scope=tt.scope.origin(l["Sale ID"]),
)

We can now plot the evolution of the sales per country over time:

![Amount per country over time](images/gifs/split-by-country.gif)

In [ ]:
session.visualize()

#### Margin

Now that the price of each product is available from the products table, we can compute the margin.

To do that, we start by computing the cost which is the quantity sold multiplied by the purchase price, summed over all the products.

In [ ]:
cost = tt.agg.sum(
    m["Quantity.SUM"] * tt.value(products_table["Purchase price"]),
    scope=tt.scope.origin(l["Product"]),
)

In [ ]:
m["Margin"] = m["Amount.SUM"] - cost

We can also define the margin rate which is the ratio of the margin by the the sold amount:

In [ ]:
m["Margin rate"] = m["Margin"] / m["Amount.SUM"]

In [ ]:
cube.query(m["Margin"], m["Margin rate"], levels=[l["Product"]])

Let's use this margin rate to do a _Top 10_ filter to see the products with the best rate.

Note that you don't need to put the rate measure and the product level in the pivot table to apply the filter.

![top10 filter on the margin rate](images/gifs/top10-margin.gif)

In [ ]:
session.visualize()

#### Cumulative sum over time

A cumulative sum is the partial sum of the data up to the current value.
For instance, a cumulative sum over time can be used to show how some measure changes over time.

In [ ]:
m["Cumulative amount"] = tt.agg.sum(
    m["Amount.SUM"], scope=tt.scope.cumulative(l["Date"])
)

![Cumulative amount](images/gifs/cumulative-amount.png)

In [ ]:
session.visualize()

#### Average per table

Aggregations can also be combined.
For instance, we can sum inside a **Shop**: then take the average of this to see how much a table sales on average:

In [ ]:
m["Average amount per shop"] = tt.agg.mean(
    m["Amount.SUM"], scope=tt.scope.origin(l["Shop"])
)

In [ ]:
cube.query(
    m["Average amount per shop"], include_totals=True, levels=[l["Sub category"]]
)

### Multilevel hierarchies

So far, all our hierarchies only had one level but it's best to regroup attributes with a parent-child relationship in the same hierarchy.

For example, we can group the **Category**, **SubCategory** and **Product ID** levels into a **Product** hierarchy:

In [ ]:
h["Product"] = [l["Category"], l["Sub category"], l["Product"]]

And let's remove the old hierarchies:

In [ ]:
del h["Category"]
del h["Sub category"]

In [ ]:
h

We can also do it with **City**, **State or Region** and **Country** to build a **Geography** hierarchy.

Note that instead of using existing levels you can also define a hierarchy with the fields of the table the levels will be based on:

In [ ]:
h["Geography"] = [
    shops_table["Country"],
    shops_table["State or region"],
    shops_table["City"],
]
del h["Country"]
del h["State or region"]
del h["City"]

As we are restructuring the hierarchies, let's use this opportunity to also change the dimensions.

A dimension regroups hierarchies of the same concept.

To keep it simple here, we will simply move the new **Geography** hierarchy to its own dimension:

In [ ]:
h["Geography"].dimension = "Location"
h

With that, we can define new measures taking advantage of the multilevel structure.
For instance, we can create a measure indicating how much a product contributes to its subcategory:

In [ ]:
m["Parent category amount"] = tt.parent_value(
    m["Amount.SUM"], degrees={h[("Products", "Product")]: 1}
)

In [ ]:
m["Percent of parent amount"] = m["Amount.SUM"] / m["Parent category amount"]

![Percent of parent](images/gifs/percent-of-parent.gif)

In [ ]:
session.visualize()

## Polishing the cube

### Deleting or hiding measures

Some measures have been automatically created from numeric fields but are not useful.
For instance, **Unit Price.SUM** does not really make sense as we never want to sum the unit prices.
We can delete it:

In [ ]:
del m["Unit price.SUM"]

Other measures have been used while building the project only as intermediary steps but are not useful to the end users in the application.
We can hide them from the UI (they will remain accessible in Python):

In [ ]:
m["Parent category amount"].visible = False

### Measure folders

Measures can be rearranged into folders.

In [ ]:
for measure in [
    m["Amount.MEAN"],
    m["Amount.SUM"],
    m["Average amount per shop"],
    m["Cumulative amount"],
    m["Percent of parent amount"],
]:
    measure.folder = "Amount"

In [ ]:
m

### Measure formatters

Some measures can be formatted for a nicer display.
Classic examples of this is changing the number of decimals or adding a percent or a currency symbol.

Let's do this for our percent of parent amount and margin rate:

#### Before

In [ ]:
cube.query(m["Percent of parent amount"], m["Margin rate"], levels=[l["Category"]])

In [ ]:
m["Percent of parent amount"].formatter = "DOUBLE[0.00%]"
m["Margin rate"].formatter = "DOUBLE[0.00%]"

#### After

In [ ]:
cube.query(m["Percent of parent amount"], m["Margin rate"], levels=[l["Category"]])

## Simulations

Simulations are a way to compare several scenarios and do what-if analysis.
This helps understanding how changing the source data or a piece of the model impact the key indicators.

In atoti, the data model is made of measures chained together.
A simulation can be seen as changing one part of the model, either its source data or one of its measure definitions, and then evaluating how it impacts the following measures.

### Source simulation

Let's start by changing the source.
With pandas or Spark, if you want to compare two results for a different versions of the entry dataset you have to reapply all the transformations to your dataset.
With atoti, you only have to provide the new data and all the measures will be automatically available for both versions of the data.

We will create a new scenario using pandas to modify the original dataset.

In [ ]:
import pandas as pd

For instance, we can simulate what would happen if we had managed to purchase some products at a cheaper price.

In [ ]:
products_df = pd.read_csv("data/products.csv")
products_df.head()

In [ ]:
better_prices = {
    "TAB_0": 180.0,
    "TAB_1": 250.0,
    "CHA_2": 40.0,
    "BED_3": 110.0,
    "BED_4": 210.0,
}

In [ ]:
for product, purchase_price in better_prices.items():
    products_df.loc[
        products_df["Product"] == product, "Purchase price"
    ] = purchase_price
products_df.head()

We can now load this new dataframe into a new scenarios of the product table.

In [ ]:
products_table.scenarios["Cheaper purchase prices"].load_pandas(products_df)

The session now has two scenarios and the only differences between them are the lines corresponding to the products with better prices, everything else is shared between the scenarios and has not been duplicated: source scenarios in atoti are memory-efficient.

<img alt="Source simulation" src="images/olap/source-simulation.svg" width="750" />

Using the **Source Simulation** hierarchy we can display the margin of the scenario and compare it to the base case.

![Source simulation comparison](images/gifs/source-simulation.gif)

In [ ]:
session.visualize()

Note that all the existing measures are immediately available on the new data.
For instance, the margin rate still exists, and we can see that in this scenario we would have a better margin for the Furniture products.

![Margin rate per product category and scenario](images/gifs/margin-rate-per-scenario.gif)

In [ ]:
session.visualize()

### Parameter simulations

The other simulation technique is to create a parameter measure whose value can be changed for some coordinates.

When creating the simulation, you can choose at which granularity the modification applies.
For instance we can create a parameter measure whose value will change depending on the country.
Doing that, we can answer questions such as "What happens if there is a crisis in France and we sell 20% less?"

In [ ]:
country_simulation = cube.create_parameter_simulation(
    "Country Simulation",
    measure_name="Country parameter",
    default_value=1.0,
    levels=[l["Country"]],
)

This has created a measure named **Country parameter** and added it to the cube. For now, its value is `1` everywhere, but using the `country_simulation` we can change that.

By adding values in the table you can change the value of the parameter measure depending on the levels used in the simulation and the scenario.

In [ ]:
country_simulation += ("France Crisis", "France", 0.80)
country_simulation.head()

Let's replace the existing **Quantity.SUM** and **Amount.SUM** measures with new ones using the parameter measure from the simulation.

In [ ]:
m["Quantity.SUM"] = tt.agg.sum(
    tt.agg.sum(sales_table["Quantity"]) * m["Country parameter"],
    scope=tt.scope.origin(l["Country"]),
)
m["Amount.SUM"] = tt.agg.sum(
    tt.agg.sum(sales_table["Unit price"] * sales_table["Quantity"])
    * m["Country parameter"],
    scope=tt.scope.origin(l["Country"]),
)

We can query the cube using the new **Country Simulation** level to compare the quantity and amount between the base case and our new scenario:

In [ ]:
cube.query(
    m["Quantity.SUM"],
    m["Amount.SUM"],
    include_totals=True,
    levels=[l["Country Simulation"], l["Country"]],
)

Here for example, as the amount has been modified, the measures depending on it such as the cumulative amount are also impacted:

In [ ]:
cube.query(m["Cumulative amount"], levels=[l["Country Simulation"], l["Country"]])

Let's try adding a different scenario:

In [ ]:
country_simulation += ("US boost", "USA", 1.15)

In [ ]:
cube.query(m["Quantity.SUM"], levels=[l["Country Simulation"], l["Country"]])

The two scenarios can be visualized in the same widget:

![Cumulative amount per scenario](images/gifs/cumulative-amount-per-scenario.gif)

In [ ]:
session.visualize()

Finally, we can even combine the different simulations (the source one and the measure one) to create a matrix of scenarios:

![Matrix of scenarios](images/gifs/scenarios-matrix.gif)

In [ ]:
session.visualize()

## Going further

You've learned all the basics to build a project with atoti, from the concept of multidimensional analysis to powerful simulations.

We now encourage you to try the library with your own data.
You can also start to learn more advanced features such as [session configuration](../lib/atoti.config.rst), [custom endpoints](../lib/atoti.session.rst#atoti.session.Session.endpoint), [querying](../lib/atoti.query.rst), and [arrays](../lib/atoti.array.rst).